In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

from pdflight import in_task, shift_datetime

sns.set(rc={"figure.figsize": (12, 8)})


In [ ]:
with open("store.pkl", "rb") as f:
    stored = pickle.load(f)
    md, fl, th = stored["md"], stored["fl"], stored["th"]
display(md.competition.unique())

competition = md.index[
    md.competition == "ssm2017"
]
if True:
    md = md.loc[competition]
    fl = fl.loc[competition]
    th = th.loc[competition]

In [ ]:
fl["te"] = fl.alt + fl.gsp**2 / 2 / 127.131

In [ ]:
tt = th.loc[in_task(th, md) & (th.dt.total_seconds() >= 120)]

In [ ]:
df = tt.reset_index()
df["h_enter"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].alt.values
df["v_enter"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].gsp.values
df["e_enter"] = df.h_enter + df.v_enter**2 / 2 / 127.131
df["exit"] = df.datetime + df.duration
df["h_exit"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].alt.values
df["v_exit"] = fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].gsp.values
df["e_exit"] = df.h_exit + df.v_exit**2 / 2 / 127.131
df["dh"] = df.h_exit - df.h_enter
df["vv"] = df.dh / df.duration.dt.total_seconds()
df["de"] = df.e_exit - df.e_enter
df["p"] = df.de / df.duration.dt.total_seconds()
if False:
    dt_stable = 40
    df["stable"] = df.datetime + pd.Timedelta(seconds=dt_stable)
    df["h_stable"] = fl.loc[
        pd.MultiIndex.from_frame(df[["flight", "stable"]])
    ].alt.values
    df["v_stable"] = fl.loc[
        pd.MultiIndex.from_frame(df[["flight", "stable"]])
    ].gsp.values
    df["e_stable"] = df.h_stable + df.v_stable**2 / 2 / 127.131
    df["dh_stable"] = df.h_stable - df.h_enter
    df["vv_stable"] = df.dh_stable / dt_stable
    df["de_stable"] = df.e_stable - df.e_enter
    df["p_stable"] = df.de_stable / dt_stable
    df["eta_vv_stable"] = df.vv_stable / df.vv
    df["eta_p_stable"] = df.p_stable / df.p
    df["loss_e_stable"] = (df.p_stable - df.p) * dt_stable
df

In [ ]:
# Berechne durchschnittliche Groundspeed für jedes Thermik-Event
thermal_gsp = []

# Iteriere direkt über die Series tt
for (flight, datetime), duration in tt.items():
    end_time = datetime + duration
    
    # Hole Flugdaten für diesen Zeitbereich
    # Verwende Boolean-Indexing statt Slicing wegen nicht-monotonem Index
    flight_data = fl.loc[flight]
    mask = (flight_data.index >= datetime) & (flight_data.index <= end_time)
    flight_segment = flight_data.loc[mask]
    
    if len(flight_segment) > 0:
        avg_gsp = flight_segment['gsp'].mean()
        thermal_gsp.append({
            'flight': flight,
            'datetime': datetime,
            'duration': duration.total_seconds(),  # Konvertiere zu Sekunden
            'avg_groundspeed': avg_gsp
        })

# Erstelle DataFrame mit Groundspeed-Durchschnitten
thermal_gsp_df = pd.DataFrame(thermal_gsp)

# Füge Groundspeed-Informationen zum Haupt-DataFrame hinzu
# Verwende nur flight und datetime für den Merge
df = df.merge(thermal_gsp_df[['flight', 'datetime', 'avg_groundspeed']], on=['flight', 'datetime'], how='left')

# Zeige die neuen Daten
df[['flight', 'datetime', 'duration', 'avg_groundspeed', 'vv']].head()

In [ ]:
cc = md.join(
    df.groupby("flight").mean(), how="inner"
)
cc.info()

In [ ]:
x = cc.select_dtypes(include=['int64', 'float64']).columns.tolist()
y = "points"
sns.pairplot(cc, kind="reg", plot_kws={"line_kws":{"color":"red"}})
pd.DataFrame([stats.linregress(cc[i], cc[y]) for i in x], index = pd.MultiIndex.from_tuples([(y, i) for i in x]))

In [ ]:
# Korrelationsmatrix erstellen
korrelationen = cc.corr(numeric_only=True)
display(korrelationen)
# Visualisierung
sns.heatmap(korrelationen, annot=True, cmap='coolwarm', )

In [ ]:
# Sample 10 thermals and plot fl.te - fl.alt using seaborn
sample_thermals = th.sample(10, random_state=42)
display(sample_thermals)

# Create plot for each sampled thermal using seaborn style
fig, axes = plt.subplots(5, 2, figsize=(15, 20), sharex=True)
axes = axes.flatten()

for i, ((flight, datetime), duration) in enumerate(sample_thermals.items()):
    end_time = datetime + duration

    # Get flight data for this thermal
    flight_data = fl.loc[flight]
    mask = (flight_data.index >= datetime) & (flight_data.index <= end_time)
    thermal_segment = flight_data.loc[mask]

    if len(thermal_segment) > 0:
        ax = axes[i]
        # Create a temporary column for the difference
        thermal_segment['te_minus_alt'] = thermal_segment['te'] - thermal_segment['alt']
        thermal_segment["dt"] = thermal_segment.index - thermal_segment.index[0]
        sns.lineplot(data=thermal_segment, x=thermal_segment.dt, y='te_minus_alt', ax=ax, label='TE - Alt')
        ax.set_title(f'Thermal {i+1}: {flight} at {datetime.strftime("%H:%M:%S")}')
        ax.set_ylabel('TE - Alt (m)')
        ax.grid(True, alpha=0.3)
        ax.legend()

        # Add vertical lines for thermal start and end
        #ax.axvline(datetime, color='red', linestyle='--', alpha=0.5)
        #ax.axvline(end_time, color='blue', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.suptitle('TE - Alt for 10 Sampled Thermals', y=1.02)
plt.show()